---

# Connect Four - AB Pruning vs A*
## Florian Frick


---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

### Game Representation

Representing the game and board as classes and methods.

In [2]:
class State:
    def __init__(self, moves):
        self.to_move = 'R' # whose turn is it
        self.utility = 0 # has someone won
        self.board = {} # (1,1):'R'
        self.moves = moves #
        
    def __lt__(self, other):
        return False # allows arbitrary comparison between states

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row,col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

    # counts how many pieces in a given column
    def pieces_in_col(self, state, col):
        '''
        Returns number of pieces in a given column
        '''
        return sum([piece[1] == col for piece in state.board]) # number of pieces already in column trying to drop into
    
    # returns move at top columns who are not full
    def legal_moves(self, state):
        return [(1,c) for c in range(1,self.ncol+1) if (self.nrow - self.pieces_in_col(state, c)) > 0]
    
    # drops piece down a column
    def move_after_fall(self, move, state):
        '''
        Returns the (row,col) a piece falls to taking move in state.
        Also ensures move is legal
        '''
        assert move in state.moves, "Tried placing in non-existent column or row."
        assert move not in state.board, "Tried placing in occupied location." # only necessary if not trying to drop from top
        
        num_in_col = self.pieces_in_col(state,move[1])
        assert (self.nrow - num_in_col) > 0, "Tried placing in full column"
        
        return (self.nrow - num_in_col, move[1])
    
    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # check whether move is legal and determine where it falls
        real_move = self.move_after_fall(move, state)
        
        
        # change board state
        new_state = cp.deepcopy(state)
        new_state.board[real_move] = state.to_move # place in column at bottom row
        new_state.to_move = 'B' if state.to_move=='R' else 'R'
        new_state.utility = self.compute_utility(real_move, state)
        
        # return new state
        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        
        
        row, col = move
        # determine if win condition met with this move
        board = cp.deepcopy(state.board)
        board[(row,col)] = state.to_move
        
        
        # check for row-wise win
        in_a_row = 0
        for c in range(col,0,-1):
            if board.get((row,c)) == state.to_move:
                in_a_row += 1
            else:
                break;
        for c in range(col+1,self.ncol+1):
            if board.get((row,c)) == state.to_move:
                in_a_row += 1
            else:
                break;

        # check for column-wise win
        in_a_col = 0
        for r in range(row,0,-1):
            if board.get((r,col)) == state.to_move:
                in_a_col += 1
            else:
                break;
        for r in range(row+1,self.nrow+1):
            if board.get((r,col)) == state.to_move:
                in_a_col += 1
            else:
                break;

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            if board.get((r,col-(row-r))) == state.to_move:
                in_a_diag1 += 1
            else:
                break;
        for r in range(row+1,self.nrow+1):
            if board.get((r,col-(row-r))) == state.to_move:
                in_a_diag1 += 1
            else:
                break;

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            if board.get((r,col+(row-r))) == state.to_move:
                in_a_diag2 += 1
            else:
                break;
        for r in range(row+1,self.nrow+1):
            if board.get((r,col+(row-r))) == state.to_move:
                in_a_diag2 += 1
            else:
                break;
                
        # win conditions
        # print(move, [in_a_row, in_a_col, in_a_diag1, in_a_diag2]) #debug
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if state.to_move=='R' else -1
        else:
            return 0 

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        if state.utility != 0:
            return True # winner
        if len(state.board) == self.nrow * self.ncol:
            return True # no moves
        return False

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        
        if player == 'R':
            return state.utility
        else:
            return -1*state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2, report_moves=False):
        '''Play a game of Connect Four!'''

        move_count=0
        while self.game_over(self.state) == False:
            move_count+=1
            # debug print
            # self.display()
            if self.state.to_move == 'R':
                move = player1(self)
            else:
                move = player2(self)
            self.state = self.result(move, self.state)
            
        if report_moves: # report how many moves to end game
            return self.utility(self.state,'R'), move_count
        
        return self.utility(self.state,'R') # 1 if player 1 wins, -1 if player 2 wins

### Random Player

Define random player agent that does a random legal move.

In [3]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    # select all moves who are not on the board
    # available_moves = [move for move in game.state.moves if move not in list(game.state.board.keys())] 
    
    # select columns that aren't full
    available_moves = game.legal_moves(game.state)
    # print(available_moves)
    # pick random move based on index in the list
    return available_moves[np.random.randint(low=0, high=len(available_moves))]

In [4]:
# 1000 games between two random players, 6x7
n=1000

results = [ConnectFour().play_game(random_player, random_player) for i in range(n)];

print("Player 1 win probability:", results.count(1)/n)
print("Player 2 win probability:", results.count(-1)/n)
print("Draw probability:", results.count(0)/n)

Player 1 win probability: 0.564
Player 2 win probability: 0.432
Draw probability: 0.004


> Random players win 55% of the time going first on a standard 6x7 board.

In [5]:
# 1000 games between two random players, 10x10
n=1000

results = [ConnectFour(nrow=10, ncol=10, nwin=6).play_game(random_player, random_player) for i in range(n)];

print("Player 1 win probability:", results.count(1)/n)
print("Player 2 win probability:", results.count(-1)/n)
print("Draw probability:", results.count(0)/n)

Player 1 win probability: 0.44
Player 2 win probability: 0.432
Draw probability: 0.128


> The state space has increased so the advantage of going first decreases, as games take longer. The more difficult win condition of 6 in a row increases the probability of draws because the players have more opportunities to interrupt each other before they get 6 in a row.

### Alpha-beta Player

Define minimax agent with alpha-beta pruning.

In [6]:
# modified from wikipedia pseudocode
def alphabeta_search(game,state,pruning=True):
    minimax_counter = 0
    prune_counter = 0
    def max_value(s,alpha,beta):
        # count expanded nodes
        if pruning:
            nonlocal prune_counter
            prune_counter += 1
        else:
            nonlocal minimax_counter
            minimax_counter += 1
        
        # termination condition
        if game.game_over(s):
            return game.utility(s,state.to_move),None
        
        value = -np.inf
        best_action = None
        
        for action in game.legal_moves(s):
            minplayer_val, min_act = min_value(game.result(action, s), alpha, beta)
            if minplayer_val > value:
                best_action = action
                value = minplayer_val
            
            # pruning
            if pruning and value >= beta:
                return value,best_action
            
            alpha = max(value,alpha)
        return value, best_action

    def min_value(s,alpha,beta):
        # count expanded nodes
        if pruning:
            nonlocal prune_counter
            prune_counter += 1
        else:
            nonlocal minimax_counter
            minimax_counter += 1

        # termination condition
        if game.game_over(s):
            return game.utility(s,state.to_move),None
        
        value = np.inf
        best_action = None
        
        for action in game.legal_moves(s):
            maxplayer_val, max_act = max_value(game.result(action, s), alpha, beta)
            if maxplayer_val < value:
                best_action = action
                value = maxplayer_val
            
            # pruning
            if pruning and value <= alpha:
                return value,best_action
            
            beta = min(value,beta)
        return value, best_action

    return max_value(state, -np.inf, np.inf), minimax_counter, prune_counter


def alphabeta_player(game):
    move, minimax_counter, prune_counter = alphabeta_search(game,game.state)
    return move[1]

In [7]:
# 10 games between an alphabeta and random player, 3x4 3 in a a row
n=100

results = [ConnectFour(nrow=3,ncol=4,nwin=3).play_game(alphabeta_player, random_player) for i in range(n)];

print("AB vs random on 3x4")
print("Player 1 win probability:", results.count(1)/n)
print("Player 2 win probability:", results.count(-1)/n)
print("Draw probability:", results.count(0)/n)

AB vs random on 3x4
Player 1 win probability: 1.0
Player 2 win probability: 0.0
Draw probability: 0.0


In [8]:
# 10 games between two alphabeta players
n=10

results = [ConnectFour(nrow=3,ncol=4,nwin=3).play_game(alphabeta_player, alphabeta_player) for i in range(n)];

print("AB vs AB on 3x4")
print("Player 1 win probability:", results.count(1)/n)
print("Player 2 win probability:", results.count(-1)/n)
print("Draw probability:", results.count(0)/n)

AB vs AB on 3x4
Player 1 win probability: 1.0
Player 2 win probability: 0.0
Draw probability: 0.0


In [9]:
# 10 games between two alphabeta players
n=10

results = [ConnectFour(nrow=3,ncol=3,nwin=3).play_game(alphabeta_player, alphabeta_player) for i in range(n)];

print("AB vs AB on 3x3")
print("Player 1 win probability:", results.count(1)/n)
print("Player 2 win probability:", results.count(-1)/n)
print("Draw probability:", results.count(0)/n)

AB vs AB on 3x3
Player 1 win probability: 0.0
Player 2 win probability: 0.0
Draw probability: 1.0


> The AB player wins against the random player every time when going first. It sometimes loses when going second due to the small board size, but rarely, and not when the board size is increased.

> AB players always draw when playing against each other on some boards, including a 3x3 board, but on others, including a 3x4 board, one player wins.

### Benefits of AB Pruning

In [10]:
# minimax vs pruning (3x4, 3 in a row)

game1 = ConnectFour(nrow=3,ncol=4,nwin=3)
game2 = ConnectFour(nrow=3,ncol=4,nwin=3)

minimax = alphabeta_search(game1, game1.state, pruning=False)
pruning = alphabeta_search(game2, game2.state, pruning=True)

print("Number of nodes expanded with pruning:", pruning[2])
print("Number of nodes expanded with minimax:", minimax[1]) # state space minus board with multiple wins
print("Ratio: %.5f" % (pruning[2]/minimax[1]))

Number of nodes expanded with pruning: 1115
Number of nodes expanded with minimax: 289869
Ratio: 0.00385


> AB-pruning expands far fewer nodes than minimax, about 0.00385 the number of nodes. Because minimax goes one move at a time, it searches every state in the state space minus ones with multiple wins. AB pruning still grows exponentially due to its nature, but it is much better than minimax because almost every state has a utility of 0, so once a win/loss is found, almost all subsequent branches get pruned. 

### A* Player

> **Heuristic:** The A* heuristic here is admissible and consistent. It scans the board state to determine how well each player is doing and can do. If the state is won, it of course returns a very high or very low heuristic value, based on which player is currently trying to find a move. Similarly, if the opponent is about to make a move winning them the game, the heuristic is very large. This takes into account disjoint sets so that separated pieces can not come together into a win for the opponent. If there are no immediate threats, the heuristic increases exponentially based on how many in a row this player has. Therefore, 1 on its own is much less valuable than 2 in a row which is much less valuable than 3 in a row, etc.

In [11]:
# Heuristic evaluates state
# return negatives if winning because A* tries to minimize heuristic as frontier's priority queue is a min-heap
def heuristic(game, state, player):
    # # if someone has won in this state
    if state.utility != 0:
        if player=='R':
            return -1*state.utility * 10000
        else:
            return state.utility * 10000
    
    # scan board, determine how many in a row for each player
    R_best = 0
    B_best = 0
    for loc,p in state.board.items():
        nums = in_a_row(game,state,loc[0],loc[1],p)
        if p=='R':
            R_best = max(R_best, max(nums))
        else:
            B_best = max(B_best, max(nums))
    
    # opponent one move from win
    if player=='R' and state.to_move=='B':
        for move in game.legal_moves(state):
                temp_state = game.result(move,state)
                if temp_state.utility != 0:
                    return 10000
    elif player=='B' and state.to_move=='R':
        for move in game.legal_moves(state):
                temp_state = game.result(move,state)
                if temp_state.utility != 0:
                    return 10000

    # exponentially increase based on number in a row
    if player=='R':
        return -1*(np.exp(R_best))
    return -1*(np.exp(B_best))

def backtrack(previous, s): 
    if s is None:
        return []
    else:
        return backtrack(previous, previous[s])+[s]

def pathcost(path, step_costs):
    cost = 0
    for s in range(len(path)-1):
        cost += step_costs[path[s]][path[s+1]]
    return cost

class Frontier_PQ:
    def __init__(self,start,cost):
        self.states = {start: cost} #lowest cost to each state visited thus far
        self.q = [(cost,start)] #priority queue ordered by cost
    
    # search states to find cost of this state
    def get_val_from_state(self, new_state):
        for s in self.states:
            if state_in_list(new_state, [s]):
                return self.states.get(s)
        return None
    
    def add(self,state,cost):
        s = self.states.get(state)
        if s is None:
            # first time reaching state
            heapq.heappush(self.q, (cost,state))
            self.states[state]=cost
        elif s > cost:
            # better path to state
            self.replace(state,cost)
        
    def pop(self):
        try: 
            s = heapq.heappop(self.q) #pop from queue
            self.states.pop(s[1]) #remove from dictionary
            return s
        except:
            # print("Popping off empty queue")
            return None #empty queue
        
    def replace(self,state,cost):
        print("replacing")
        oldCost = self.states[state]
        self.states[state] = cost #replace cost in dictionary
        #replace in queue
        self.q.remove((oldCost,state))
        self.q.append((cost,state))
        heapq.heapify(self.q)
        
def in_a_row(game, state, row, col, player):
    board = state.board
    in_a_row = 0
    for c in range(col,0,-1):
        if board.get((row,c)) == player:
            in_a_row += 1
        else:
            break;
    for c in range(col+1,game.ncol+1):
        if board.get((row,c)) == player:
            in_a_row += 1
        else:
            break;

    in_a_col = 0
    for r in range(row,0,-1):
        if board.get((r,col)) == player:
            in_a_col += 1
        else:
            break;
    for r in range(row+1,game.nrow+1):
        if board.get((r,col)) == player:
            in_a_col += 1
        else:
            break;

    in_a_diag1 = 0
    for r in range(row,0,-1):
        if board.get((r,col-(row-r))) == player:
            in_a_diag1 += 1
        else:
            break;
    for r in range(row+1,game.nrow+1):
        if board.get((r,col-(row-r))) == player:
            in_a_diag1 += 1
        else:
            break;

    in_a_diag2 = 0
    for r in range(row,0,-1):
        if board.get((r,col+(row-r))) == player:
            in_a_diag2 += 1
        else:
            break;
    for r in range(row+1,game.nrow+1):
        if board.get((r,col+(row-r))) == player:
            in_a_diag2 += 1
        else:
            break;

    return [in_a_row, in_a_col, in_a_diag1, in_a_diag2]


# identify difference between the two board states
def move_between_two_states(s0, s1):
    # ensure one move probably separates the states
    if len(s0.board) != len(s1.board)-1:
        print("State 1 does not follow state 2.")
        return False

    ret_loc = None
    for loc in s1.board.keys():
        if loc not in s0.board.keys():
            if ret_loc is not None:
                print("More than one difference between states.")
            ret_loc = loc # difference between the two states
    return ret_loc
    
# returns true if state s in list of states l
def state_in_list(s, l):
    # compare each state in list
    for s2 in l:
        if s2.to_move == s.to_move and s2.utility == s.utility and len(s2.board) == len(s.board): # preliminary checks
            ret_val = True # assume boards are the same
            for loc,p in s2.board.items():
                if s.board.get(loc) != p:
                    ret_val = False # proved boards are not the same
                    break
            if ret_val == True:
                return True # boards were the same
    return False # no equivalent state found
    
# Given a game (in a state), perform A* to find win, return move to make from that state
def astar(game):
    # initialize
    player = game.state.to_move
    start = game.state
    pq = Frontier_PQ(start,heuristic(game,start,player))
    explored = []
    prev = {start: None}
    
    # find path
    while True:
        # explore next state
        v = pq.pop()
        
        if v is None:
            # print("could not find goal")
            return game.legal_moves(start)[0] # could not find goal (act randomly, for now)
        
        current_cost = v[0]
        current_state = v[1]
        explored.append(current_state)
        
        
        if game.utility(current_state, player) == 1: # found win
            path = backtrack(prev, current_state) # dictionary of paths and final state
            if len(path) < 2:
                print("Already at goal, no move to be made")
                return False
            move = move_between_two_states(path[0], path[1]) # get first move to make from start(path[0])
            return move # best move
        
        for move in game.legal_moves(current_state):
            new_state = game.result(move,current_state)
            new_in_explored = state_in_list(new_state, explored)
            new_in_pq = state_in_list(new_state, pq.states)
            if new_in_explored == False and new_in_pq == False: # new state not been explored, and not in frontier
                prev[new_state] = current_state
                pq.add(new_state, 1 + current_cost + heuristic(game,new_state,player)) #edge weight current to new state and distance from new state to goal
            elif new_in_pq == True: # new state in frontier
                # print("Has visited state before", new_state)
                if (1 + current_cost + heuristic(game,new_state,player)) < pq.get_val_from_state(new_state): # better path to new_state
                    prev[new_state] = v[1]
                    pq.replace(new_state, 1 + current_cost + heuristic(game,new_state,player)) #edge current to new state and distance from new state to goal

def astar_player(game):
    move = astar(game)
    # print(move)
    return move

### A* vs AB vs Random

> Against a random player, A* wins almost game, and when going second more consistently than AB pruning. The games took 11.35 moves on average, because A* blocks the random player's possible wins, even if it won't take advantage, and gets randomly blocked. Therefore, the bigger the board size, the fewer moves it takes to win, and the less likely it is for a random player to be able to win or draw as it will make mistakes more often.

> Against an AB-player, A* wins every game. The games took 5 moves on average, because the AB pruning player only acts against what an AB pruning player would do, not any other kind of player. Also, when no moves are winning, it picks the first option it finds because very losing state has the same utility.

> Against itself, A* ties every game. The games took 42 moves on average, because both A* players block each other's wins every time, and do not have a complex enough heuristic to establish tactics and strategies like base-inverse, claim-even. The smaller the board, the bigger an advantage the first player has because they have tempo. 
 
> The A* player seems much better than the AB-player for several reasons. Most importantly, A* runs much faster than AB-pruning because it does not exponentially explore every possible path to make a move. A* uses a heuristic, which runs quickly and as a bonus, can be improved more flexibly. A* also performs better than AB-pruning because AB-pruning only acts optimally against itself. If its opponent deviates from what the min-player would do, the action it chose was likely suboptimal, and is thus open to be exploited by other strategies, such as A*.

> The A* algorithm could be expanded in the future to implement more complex heuristics. A heuristic that accounts for tactics, zugzswang, base-inverse, and claim-even rules would be difficult to implement but very effective in looking ahead and understanding which threats are relevant, and what can be ignored to get a better position.

In [12]:
# A* vs random (6x7)
n=100

results = [ConnectFour(nrow=6,ncol=7,nwin=4).play_game(astar_player, random_player, report_moves=True) for i in range(n)];
winners = [result[0] for result in results]
move_counts = [result[1] for result in results]

print("Player 1 win probability:", winners.count(1)/n)
print("Player 2 win probability:", winners.count(-1)/n)
print("Draw probability:", winners.count(0)/n)
print("Avg # moves:", sum(move_counts)/n)

Player 1 win probability: 0.99
Player 2 win probability: 0.01
Draw probability: 0.0
Avg # moves: 11.35


In [13]:
# A* vs AB (3x4)
n=10

results = [ConnectFour(nrow=3, ncol=4,nwin=3).play_game(astar_player, alphabeta_player, report_moves=True) for i in range(n)];
winners = [result[0] for result in results]
move_counts = [result[1] for result in results]

print("Player 1 win probability:", winners.count(1)/n)
print("Player 2 win probability:", winners.count(-1)/n)
print("Draw probability:", winners.count(0)/n)
print("Avg # moves:", sum(move_counts)/n)

Player 1 win probability: 1.0
Player 2 win probability: 0.0
Draw probability: 0.0
Avg # moves: 5.0


In [14]:
# A* vs A* (6x7)
n=10

results = [ConnectFour(nrow=6, ncol=7,nwin=4).play_game(astar_player, astar_player, report_moves=True) for i in range(n)];
winners = [result[0] for result in results]
move_counts = [result[1] for result in results]

print("Player 1 win probability:", winners.count(1)/n)
print("Player 2 win probability:", winners.count(-1)/n)
print("Draw probability:", winners.count(0)/n)
print("Avg # moves:", sum(move_counts)/n)

Player 1 win probability: 0.0
Player 2 win probability: 0.0
Draw probability: 1.0
Avg # moves: 42.0


> 